In [1]:
import numpy as np
from scipy.stats import multivariate_normal
from matplotlib import pyplot as plt
from copy import copy
from datetime import datetime, timedelta

from stonesoup.functions import gm_reduce_single

from stonesoup.predictor.kalman import KalmanPredictor
from stonesoup.updater.kalman import KalmanUpdater
from stonesoup.predictor.imm import IMMPredictor
from stonesoup.updater.imm import IMMUpdater
from stonesoup.models.transition.linear import ConstantVelocity, \
    CombinedLinearGaussianTransitionModel, LinearGaussianTimeInvariantTransitionModel

from stonesoup.models.measurement.linear import LinearGaussian

from stonesoup.types.state import GaussianState
from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.detection import Detection
from stonesoup.types.state import StateVector, CovarianceMatrix, \
    GaussianMixtureState, WeightedGaussianState
from stonesoup.types.track import Track

from stonesoup.simulator.simple import SingleTargetGroundTruthSimulator

from matplotlib.patches import Ellipse


In [2]:
from bayou.datastructures import Gaussian as EMGPB2Gaussian
from bayou.datastructures import GMM as EMGPB2GMM
from bayou.datastructures import GMMSequence as EMGPB2GMMSequence
from bayou.models import LinearModel as EMGPB2LinearModel
from bayou.models import ConstantVelocity as EMGPB2ConstantVelocity
from bayou.expmax.skf import SKF

Plot function

In [3]:
def plot_cov_ellipse(cov, pos, nstd=2, ax=None, **kwargs):
    """
    Plots an `nstd` sigma error ellipse based on the specified covariance
    matrix (`cov`). Additional keyword arguments are passed on to the
    ellipse patch artist.
    Parameters
    ----------
        cov : The 2x2 covariance matrix to base the ellipse on
        pos : The location of the center of the ellipse. Expects a 2-element
            sequence of [x0, y0].
        nstd : The radius of the ellipse in numbers of standard deviations.
            Defaults to 2 standard deviations.
        ax : The axis that the ellipse will be plotted on. Defaults to the
            current axis.
        Additional keyword arguments are pass on to the ellipse patch.
    Returns
    -------
        A matplotlib ellipse artist
    """

    def eigsorted(cov):
        vals, vecs = np.linalg.eigh(cov)
        order = vals.argsort()[::-1]
        return vals[order], vecs[:, order]

    if ax is None:
        ax = plt.gca()

    vals, vecs = eigsorted(cov)
    theta = np.degrees(np.arctan2(*vecs[:, 0][::-1]))

    # Width and height are "full" widths, not radius
    width, height = 2 * nstd * np.sqrt(vals)
    ellip = Ellipse(xy=pos, width=width, height=height, angle=theta,
                    **kwargs)

    ax.add_artist(ellip)
    return ellip


Groundtruth and detections

In [4]:
gt_transition_model_1 = CombinedLinearGaussianTransitionModel(
                        (ConstantVelocity(2.0),
                         ConstantVelocity(2.0)))

gt_track = []
gt_time = []
iter_model_1 = 60
iter_model_2 = 140

timestamp_init = datetime.now()
state_init_1 = GaussianState(StateVector([[0], [0], [0], [0]]),
                           CovarianceMatrix(np.diag([0.0, 0.0, 0.0, 0.0])),
                           timestamp=timestamp_init)
gt_generator_1 = SingleTargetGroundTruthSimulator(gt_transition_model_1, state_init_1, number_steps=iter_model_1)

for time, gnd_paths in gt_generator_1.groundtruth_paths_gen():
    gnd_path = gnd_paths.pop()
    gt_track.append(gnd_path.state)
    gt_time.append(time)

gt_transition_model_2 = CombinedLinearGaussianTransitionModel(
                    (ConstantVelocity(0.5),
                    ConstantVelocity(0.5)))
state_init_2 = GaussianState(gt_track[-1].state_vector,
                           CovarianceMatrix(np.diag([0.0, 0.0, 0.0, 0.0])),
                           timestamp=gt_time[-1])
gt_generator_2 = SingleTargetGroundTruthSimulator(gt_transition_model_2, state_init_2, number_steps=iter_model_2)
for time, gnd_paths in gt_generator_2.groundtruth_paths_gen():
    gnd_path = gnd_paths.pop()
    gt_track.append(gnd_path.state)
    gt_time.append(time)

detection_track = []
mesurement_noise_1 = np.diag([0.5, 0.5])
measurement_model_1 = LinearGaussian(ndim_state=4, mapping=[0, 2], noise_covar=mesurement_noise_1)
for i in range(iter_model_1):
    m_ = Detection(measurement_model_1.function(gt_track[i].state_vector, measurement_model_1.rvs(1)), gt_time[i])
    detection_track.append(m_)

mesurement_noise_2 = np.diag([0.1, 0.1])
measurement_model_2 = LinearGaussian(ndim_state=4, mapping=[0, 2], noise_covar=mesurement_noise_2)
for i in range(iter_model_2):
    ii = i + iter_model_1
    m_ = Detection(measurement_model_2.function(gt_track[ii].state_vector, measurement_model_2.rvs(1)), gt_time[ii])
    detection_track.append(m_)

# print(gt_track)
# print(detection_track)
# print(gt_time)

Filter model. Testing without EMGPB2 estimated parameters.

In [5]:
ft_transition_model_1 = CombinedLinearGaussianTransitionModel(
                        (ConstantVelocity(0.1),
                         ConstantVelocity(0.1)))
ft_transition_model_2 = CombinedLinearGaussianTransitionModel(
                        (ConstantVelocity(5.0),
                         ConstantVelocity(5.0)))
ft_measurement_noise = np.diag([2.0, 2.0])

ft_measurement_model = LinearGaussian(ndim_state=4, mapping=[0, 2],
                                   noise_covar=ft_measurement_noise)

ft_model_transition_matrix = np.array([[0.5, 0.5],
                                       [0.5, 0.5]])
ft_predictor_1 = KalmanPredictor(ft_transition_model_1)
ft_predictor_2 = KalmanPredictor(ft_transition_model_2)
ft_imm_predictor = IMMPredictor([ft_predictor_1, ft_predictor_2], ft_model_transition_matrix)

ft_updater = KalmanUpdater(ft_measurement_model)
ft_imm_updater = IMMUpdater([ft_updater, ft_updater], ft_model_transition_matrix)

ft_state_init = WeightedGaussianState(StateVector([[0], [0], [0], [0]]),
                                      CovarianceMatrix(np.diag([5.0, 5.0, 5.0, 5.0])),
                                      timestamp=timestamp_init,
                                      weight=0.5)

prior1 = copy(ft_state_init)
prior = GaussianMixtureState([prior1, prior1])
track = Track([copy(prior)])
track_error = []

for i in range(1, len(detection_track)-1):
    measurement = detection_track[i]
    # State prediction
    prediction = ft_imm_predictor.predict(track.state, timestamp=gt_time[i])
    # Measurement prediction
    meas_prediction = ft_imm_updater.predict_measurement(prediction)
    # State update
    hyp = SingleHypothesis(prediction, measurement)
    prior = ft_imm_updater.update(hyp)
    track.append(prior)
    track_error.append(ft_measurement_model.function(track[i].state_vector, np.zeros((2,1))) - ft_measurement_model.function(gt_track[i].state_vector, np.zeros((2,1))))
    '''
    fig, (ax1, ax2) = plt.subplots(2,1)
    # Generate plots
    ax1.cla()
    ax2.cla()
    # PLot true trajectory
    data = np.array([state.state_vector for state in gt_track])
    ax1.plot(data[:, 0], data[:, 2], 'b-')
    # PLot estimated trajectory
    data = np.array([state.state_vector for state in track.states])
    ax1.plot(data[:, 0], data[:, 2], 'r-')
    # Plot innovation covariance
    plot_cov_ellipse(meas_prediction.covar,
                     meas_prediction.mean, edgecolor='b',
                     facecolor='none', ax=ax1)
    # Plot estimated covariance
    plot_cov_ellipse(track.state.covar[[0, 2], :][:, [0, 2]],
                     track.state.mean[[0, 2], :], edgecolor='r',
                     facecolor='none', ax=ax1)
    # Visualise model weights
    ax2.bar([1,2], prior.weights.ravel())
    plt.pause(0.0001)
    '''

track_error = np.asarray(track_error)
track_error = np.squeeze(track_error)
rmse_tmp = []
for track_error_ in track_error:
    rmse_tmp.append(track_error_ @ track_error_.T)
rmse = np.sqrt(np.mean(np.array(rmse_tmp)))
print("RMSE: " + str(rmse))

RMSE: 0.6941174585897624


Filter model. Testing with correct parameters.

In [6]:
model_transition_matrix = np.array([[0.5, 0.5],
                                    [0.5, 0.5]])

predictor_1 = KalmanPredictor(gt_transition_model_1)
predictor_2 = KalmanPredictor(gt_transition_model_2)
imm_predictor = IMMPredictor([predictor_1, predictor_2],
                             model_transition_matrix)

updater_1 = KalmanUpdater(measurement_model_1)
updater_2 = KalmanUpdater(measurement_model_2)
imm_updater = IMMUpdater([updater_1, updater_2], model_transition_matrix)

prior1 = copy(ft_state_init)
prior = GaussianMixtureState([prior1, prior1])
track = Track([copy(prior)])
track_error = []

for i in range(1, len(detection_track)-1):
    measurement = detection_track[i]
    # State prediction
    prediction = imm_predictor.predict(track.state, timestamp=gt_time[i])
    # Measurement prediction
    meas_prediction = imm_updater.predict_measurement(prediction)
    # State update
    hyp = SingleHypothesis(prediction, measurement)
    prior = imm_updater.update(hyp)
    track.append(prior)
    track_error.append(ft_measurement_model.function(track[i].state_vector, np.zeros((2,1))) - ft_measurement_model.function(gt_track[i].state_vector, np.zeros((2,1))))
    # print(ft_measurement_model.function(track[i].state_vector, np.zeros((2,1))))
    # print(ft_measurement_model.function(gt_track[i].state_vector, np.zeros((2,1))))
    # print('----------------------')
    
    '''
    fig, (ax1, ax2) = plt.subplots(2,1)
    # Generate plots
    ax1.cla()
    ax2.cla()
    # PLot true trajectory
    data = np.array([state.state_vector for state in gt_track])
    ax1.plot(data[:, 0], data[:, 2], 'b-')
    # PLot estimated trajectory
    data = np.array([state.state_vector for state in track.states])
    ax1.plot(data[:, 0], data[:, 2], 'r-')
    # Plot innovation covariance
    plot_cov_ellipse(meas_prediction.covar,
                     meas_prediction.mean, edgecolor='b',
                     facecolor='none', ax=ax1)
    # Plot estimated covariance
    plot_cov_ellipse(track.state.covar[[0, 2], :][:, [0, 2]],
                     track.state.mean[[0, 2], :], edgecolor='r',
                     facecolor='none', ax=ax1)
    # Visualise model weights
    ax2.bar([1,2], prior.weights.ravel())
    plt.pause(0.0001)
    '''

track_error = np.asarray(track_error)
track_error = np.squeeze(track_error)
rmse_tmp = []
for track_error_ in track_error:
    rmse_tmp.append(track_error_ @ track_error_.T)
rmse = np.sqrt(np.mean(np.array(rmse_tmp)))
print("RMSE: " + str(rmse))

RMSE: 0.6169310201320911


EMGPB2

In [7]:
F = np.asarray([
        [1, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 1],
        [0, 0, 0, 1]
    ])
H = np.asanyarray([
    [1, 0, 0, 0],
    [0, 0, 1, 0]
])
def get_Q(Q_sig, dt=1):
    Q = Q_sig * np.asarray([
        [(1/3)*np.power(dt, 3), (1/2)*np.power(dt, 2), 0, 0],
        [(1/2) * np.power(dt, 2), dt, 0, 0],
        [0, 0, (1/3)*np.power(dt, 3), (1/2)*np.power(dt, 2)],
        [0, 0, (1/2) * np.power(dt, 2), dt]
    ])
    return Q

In [11]:
g1 = EMGPB2Gaussian(np.zeros([4, 1]), 10.0*np.eye(4))
g2 = EMGPB2Gaussian(np.zeros([4, 1]), 10.0*np.eye(4))
initial_gmm_state = EMGPB2GMM([g1, g2])

# measurements = 5 * np.random.randn(200, 2, 1) + 1

detection_track_EMGPB2 = [ele.state_vector for ele in detection_track]
gmmsequence = EMGPB2GMMSequence(np.float64(detection_track_EMGPB2), initial_gmm_state)

m1 = EMGPB2LinearModel(F, get_Q(3.0), H, 0.5*np.eye(2))
m2 = EMGPB2LinearModel(F, get_Q(0.05), H, 0.5*np.eye(2))
initial_models = [m1, m2]

Z = np.ones([2, 2]) / 2
dataset = [gmmsequence]

new_models, Z, dataset, LL = SKF.EM(dataset, initial_models, Z,
                                    max_iters=200, threshold=0.00001, learn_H=True, learn_R=True,
                                    learn_A=False, learn_Q=True, learn_init_state=False, learn_Z=True,
                                    keep_Q_structure=False, diagonal_Q=False, wishart_prior=False)

-783.3643628955554
model -- 0 new_Q: 
[[ 0.93282205  0.82856893 -0.0061761  -0.01890245]
 [ 0.82856893  1.71048476  0.02215305  0.01457029]
 [-0.0061761   0.02215305  1.12170702  1.11704934]
 [-0.01890245  0.01457029  1.11704934  2.14842719]]
model -- 1 new_Q: 
[[ 8.01781085e-01  6.47109543e-01  4.97098226e-04 -4.02102215e-03]
 [ 6.47109543e-01  1.35861801e+00  8.95016437e-04 -1.28171536e-04]
 [ 4.97098226e-04  8.95016437e-04  7.47699399e-01  6.03912225e-01]
 [-4.02102215e-03 -1.28171536e-04  6.03912225e-01  1.26979774e+00]]
model -- 0 new_H: 
[[ 9.99809935e-01 -7.56398112e-03  9.94214932e-05 -2.83102598e-04]
 [-4.69384057e-04  1.04828488e-02  1.00009401e+00 -7.53419067e-04]]
model -- 1 new_H: 
[[ 1.00004722e+00 -1.07200070e-02  4.60094666e-05 -7.82149947e-04]
 [ 3.91432387e-04 -8.05127830e-03  9.99906298e-01  1.76661992e-03]]
model -- 0 new_R: 
[[0.57177742 0.02413214]
 [0.02413214 0.91361262]]
model -- 1 new_R: 
[[0.54089309 0.02922997]
 [0.02922997 0.6314527 ]]
new_Z: 
[[0.47145292 

-784.5127791788007
model -- 0 new_Q: 
[[1.02653471 0.66507298 0.11425469 0.00702998]
 [0.66507298 1.32681251 0.07010089 0.02758121]
 [0.11425469 0.07010089 2.78432203 0.52875581]
 [0.00702998 0.02758121 0.52875581 1.0155816 ]]
model -- 1 new_Q: 
[[1.01895914 0.65824927 0.06806556 0.00592952]
 [0.65824927 1.31371557 0.04223957 0.02167941]
 [0.06806556 0.04223957 1.61793289 0.48575945]
 [0.00592952 0.02167941 0.48575945 0.97943113]]
model -- 0 new_H: 
[[ 9.99774406e-01 -7.11511617e-03  1.13645891e-04 -1.15632339e-03]
 [-4.03895725e-04  9.18975452e-03  1.00010504e+00 -3.63646582e-03]]
model -- 1 new_H: 
[[ 9.99824658e-01 -8.34932797e-03  1.02712569e-04 -9.63232675e-04]
 [ 1.54677878e-04 -2.17746855e-03  9.99957319e-01  8.48309060e-04]]
model -- 0 new_R: 
[[0.14093134 0.0296838 ]
 [0.0296838  0.50510916]]
model -- 1 new_R: 
[[0.14047315 0.02773789]
 [0.02773789 0.41385233]]
new_Z: 
[[0.42327479 0.57170008]
 [0.42327479 0.57170008]]
-----------------------------------------------
-782.02126

-756.8796356451292
model -- 0 new_Q: 
[[1.03974671 0.74840329 0.4554298  0.05125855]
 [0.74840329 1.43611817 0.37897129 0.05543042]
 [0.4554298  0.37897129 7.8351029  0.95861276]
 [0.05125855 0.05543042 0.95861276 0.84537864]]
model -- 1 new_Q: 
[[0.95479188 0.6735355  0.08863334 0.01973345]
 [0.6735355  1.33745404 0.06510824 0.02745673]
 [0.08863334 0.06510824 1.40023917 0.38811741]
 [0.01973345 0.02745673 0.38811741 0.70492445]]
model -- 0 new_H: 
[[ 9.99681498e-01 -5.29287410e-03  1.34873839e-04 -1.39122319e-03]
 [-3.57902677e-04  1.00029323e-02  1.00008607e+00 -3.14478202e-03]]
model -- 1 new_H: 
[[ 9.99827683e-01 -9.36870329e-03  1.07918052e-04 -1.23081238e-03]
 [ 2.20503901e-05 -2.86565229e-04  1.00000635e+00 -1.59041707e-03]]
model -- 0 new_R: 
[[0.08886747 0.01865276]
 [0.01865276 0.22958707]]
model -- 1 new_R: 
[[0.08708013 0.01725518]
 [0.01725518 0.20634623]]
new_Z: 
[[0.21114584 0.78382903]
 [0.21114584 0.78382903]]
-----------------------------------------------
-751.36698

-709.3348876126734
model -- 0 new_Q: 
[[  1.4392691    0.95043042   5.93125293   0.77600297]
 [  0.95043042   1.41670933   2.19244506   0.49622733]
 [  5.93125293   2.19244506 109.04405876   2.67102343]
 [  0.77600297   0.49622733   2.67102343   2.81756222]]
model -- 1 new_Q: 
[[ 0.96273687  0.69465876  0.06021652 -0.01076465]
 [ 0.69465876  1.37486395 -0.00593028 -0.05002483]
 [ 0.06021652 -0.00593028  0.93313355  0.44186061]
 [-0.01076465 -0.05002483  0.44186061  0.76826073]]
model -- 0 new_H: 
[[ 9.99359511e-01 -4.07968347e-03  1.46360150e-04  7.10145535e-03]
 [-1.32636648e-03  3.82176067e-02  1.00026209e+00 -5.84312674e-03]]
model -- 1 new_H: 
[[ 9.99824747e-01 -1.01434387e-02  1.16054675e-04 -1.77275722e-03]
 [ 8.13179897e-05 -2.70855263e-03  1.00000442e+00 -2.27860727e-03]]
model -- 0 new_R: 
[[0.07107046 0.01131122]
 [0.01131122 0.12265074]]
model -- 1 new_R: 
[[0.06875784 0.01136961]
 [0.01136961 0.09868489]]
new_Z: 
[[0.01725764 0.97771724]
 [0.01725764 0.97771724]]
----------

-705.0244193534337
model -- 0 new_Q: 
[[  1.44392665   0.81388797   8.12100913   0.72120133]
 [  0.81388797   1.24577187   1.11150486   0.54245857]
 [  8.12100913   1.11150486 147.0727708    1.07602148]
 [  0.72120133   0.54245857   1.07602148   2.64712842]]
model -- 1 new_Q: 
[[ 0.9710378   0.70557655  0.07609451 -0.01253278]
 [ 0.70557655  1.38641827 -0.02204975 -0.07744991]
 [ 0.07609451 -0.02204975  0.93890671  0.52865051]
 [-0.01253278 -0.07744991  0.52865051  0.88730619]]
model -- 0 new_H: 
[[ 9.99771762e-01 -1.90482623e-02  2.23830076e-05  1.51099906e-02]
 [-1.90553012e-03  4.62613289e-02  1.00041118e+00 -9.39937307e-03]]
model -- 1 new_H: 
[[ 9.99809136e-01 -1.02546015e-02  1.23149531e-04 -2.05966513e-03]
 [ 1.01258637e-04 -3.50240857e-03  1.00000422e+00 -2.57431631e-03]]
model -- 0 new_R: 
[[0.06624668 0.00477747]
 [0.00477747 0.1027488 ]]
model -- 1 new_R: 
[[0.06033878 0.0106158 ]
 [0.0106158  0.07253874]]
new_Z: 
[[0.01263046 0.98234442]
 [0.01263046 0.98234442]]
----------

-704.0799787173314
model -- 0 new_Q: 
[[  1.47697455   0.74380578   9.00658958   0.62470175]
 [  0.74380578   1.20409905   0.42156643   0.4585949 ]
 [  9.00658958   0.42156643 154.37218891   0.89185166]
 [  0.62470175   0.4585949    0.89185166   2.36349125]]
model -- 1 new_Q: 
[[ 0.97520559  0.7098395   0.08084536 -0.01379392]
 [ 0.7098395   1.38909829 -0.02403747 -0.0817852 ]
 [ 0.08084536 -0.02403747  0.94908366  0.55200849]
 [-0.01379392 -0.0817852   0.55200849  0.92094276]]
model -- 0 new_H: 
[[ 1.00011062e+00 -2.81434458e-02 -6.25659580e-05  1.85323434e-02]
 [-1.90002785e-03  4.55730307e-02  1.00043637e+00 -1.23759572e-02]]
model -- 1 new_H: 
[[ 9.99808964e-01 -1.06231267e-02  1.25415140e-04 -2.16387199e-03]
 [ 1.04747475e-04 -3.88503109e-03  1.00000772e+00 -2.95086579e-03]]
model -- 0 new_R: 
[[ 0.06673241 -0.00244673]
 [-0.00244673  0.09071582]]
model -- 1 new_R: 
[[0.05518737 0.00965861]
 [0.00965861 0.06016366]]
new_Z: 
[[0.0120219  0.98295298]
 [0.0120219  0.98295298]]
------

-703.6852465896238
model -- 0 new_Q: 
[[1.51831470e+00 7.20850876e-01 9.46001567e+00 6.04862135e-01]
 [7.20850876e-01 1.20175597e+00 8.99906155e-02 4.36334305e-01]
 [9.46001567e+00 8.99906155e-02 1.55742457e+02 9.41302210e-01]
 [6.04862135e-01 4.36334305e-01 9.41302210e-01 2.28630498e+00]]
model -- 1 new_Q: 
[[ 0.97841066  0.71245463  0.0827551  -0.01469885]
 [ 0.71245463  1.39062527 -0.02523578 -0.08296568]
 [ 0.0827551  -0.02523578  0.95468431  0.55945499]
 [-0.01469885 -0.08296568  0.55945499  0.93117009]]
model -- 0 new_H: 
[[ 1.00029591e+00 -3.35084690e-02 -1.07957646e-04  2.04708636e-02]
 [-1.83156546e-03  4.39590878e-02  1.00043818e+00 -1.41048372e-02]]
model -- 1 new_H: 
[[ 9.99817542e-01 -1.11324776e-02  1.24901834e-04 -2.16110496e-03]
 [ 1.01940061e-04 -4.12831262e-03  1.00001278e+00 -3.36833612e-03]]
model -- 0 new_R: 
[[ 0.06776268 -0.00712742]
 [-0.00712742  0.08248958]]
model -- 1 new_R: 
[[0.05181896 0.00864053]
 [0.00864053 0.05274421]]
new_Z: 
[[0.01193341 0.98304146]


-703.4867722708109
model -- 0 new_Q: 
[[ 1.54985008e+00  7.14230228e-01  9.69849170e+00  6.04198606e-01]
 [ 7.14230228e-01  1.20647033e+00 -6.74880801e-02  4.30794015e-01]
 [ 9.69849170e+00 -6.74880801e-02  1.55753225e+02  9.89547454e-01]
 [ 6.04198606e-01  4.30794015e-01  9.89547454e-01  2.26470941e+00]]
model -- 1 new_Q: 
[[ 0.98115608  0.71448013  0.08420744 -0.01506318]
 [ 0.71448013  1.39167735 -0.02581808 -0.08348532]
 [ 0.08420744 -0.02581808  0.95959018  0.56365253]
 [-0.01506318 -0.08348532  0.56365253  0.93519665]]
model -- 0 new_H: 
[[ 1.00040517e+00 -3.68258569e-02 -1.33635049e-04  2.15436784e-02]
 [-1.74850342e-03  4.19583981e-02  1.00043123e+00 -1.50500671e-02]]
model -- 1 new_H: 
[[ 9.99830931e-01 -1.17078683e-02  1.22730673e-04 -2.08876178e-03]
 [ 9.58558147e-05 -4.29190354e-03  1.00001857e+00 -3.79922426e-03]]
model -- 0 new_R: 
[[ 0.06828944 -0.00986794]
 [-0.00986794  0.07591312]]
model -- 1 new_R: 
[[0.0495038  0.00766418]
 [0.00766418 0.04773144]]
new_Z: 
[[0.01195

-703.3753626391832
model -- 0 new_Q: 
[[ 1.57134352e+00  7.14070554e-01  9.82641079e+00  6.07660691e-01]
 [ 7.14070554e-01  1.21085101e+00 -1.34211883e-01  4.30061013e-01]
 [ 9.82641079e+00 -1.34211883e-01  1.55604800e+02  1.00434630e+00]
 [ 6.07660691e-01  4.30061013e-01  1.00434630e+00  2.26018022e+00]]
model -- 1 new_Q: 
[[ 0.98358201  0.71619819  0.08563247 -0.01504201]
 [ 0.71619819  1.39242879 -0.02594346 -0.083816  ]
 [ 0.08563247 -0.02594346  0.96427956  0.56705536]
 [-0.01504201 -0.083816    0.56705536  0.93721864]]
model -- 0 new_H: 
[[ 1.00047380e+00 -3.89576243e-02 -1.48698142e-04  2.20979952e-02]
 [-1.66586455e-03  3.98551896e-02  1.00042109e+00 -1.55398681e-02]]
model -- 1 new_H: 
[[ 9.99846925e-01 -1.23111962e-02  1.19586027e-04 -1.97305643e-03]
 [ 8.82861365e-05 -4.41345721e-03  1.00002460e+00 -4.22839326e-03]]
model -- 0 new_R: 
[[ 0.06821896 -0.0113273 ]
 [-0.0113273   0.07032668]]
model -- 1 new_R: 
[[0.04783613 0.00677826]
 [0.00677826 0.04406575]]
new_Z: 
[[0.01200

-703.3079602901784
iterations: 74


Filter model. Testing with learnt parameters from EMGPB2.

In [12]:
em_transition_model_1 = LinearGaussianTimeInvariantTransitionModel(transition_matrix=new_models[0].A, covariance_matrix=new_models[0].Q)
em_transition_model_2 = LinearGaussianTimeInvariantTransitionModel(transition_matrix=new_models[1].A, covariance_matrix=new_models[1].Q)
em_measurement_noise_1 = new_models[0].R
em_measurement_noise_2 = new_models[1].R

em_measurement_model_1 = LinearGaussian(ndim_state=4, mapping=[0, 2],
                                   noise_covar=em_measurement_noise_1)
em_measurement_model_2 = LinearGaussian(ndim_state=4, mapping=[0, 2],
                                   noise_covar=em_measurement_noise_2)

em_model_transition_matrix = np.array([[0.5, 0.5],
                                       [0.5, 0.5]])
em_predictor_1 = KalmanPredictor(em_transition_model_1)
em_predictor_2 = KalmanPredictor(em_transition_model_2)
em_imm_predictor = IMMPredictor([em_predictor_1, em_predictor_2], em_model_transition_matrix)

em_updater_1 = KalmanUpdater(em_measurement_model_1)
em_updater_2 = KalmanUpdater(em_measurement_model_2)
em_imm_updater = IMMUpdater([em_updater_1, em_updater_2], em_model_transition_matrix)

em_state_init = WeightedGaussianState(StateVector([[0], [0], [0], [0]]),
                                      CovarianceMatrix(np.diag([5.0, 5.0, 5.0, 5.0])),
                                      timestamp=timestamp_init,
                                      weight=0.5)

prior1 = copy(em_state_init)
prior = GaussianMixtureState([prior1, prior1])
track = Track([copy(prior)])
track_error = []

for i in range(1, len(detection_track)-1):
    measurement = detection_track[i]
    
    # State prediction
    prediction = em_imm_predictor.predict(track.state, timestamp=gt_time[i])
    # Measurement prediction
    meas_prediction = em_imm_updater.predict_measurement(prediction)
    # State update
    hyp = SingleHypothesis(prediction, measurement)
    prior = em_imm_updater.update(hyp)
    track.append(prior)
    track_error.append(em_measurement_model_1.function(track[i].state_vector, np.zeros((2,1))) - em_measurement_model_1.function(gt_track[i].state_vector, np.zeros((2,1))))
    
    '''
    fig, (ax1, ax2) = plt.subplots(2,1)
    # Generate plots
    ax1.cla()
    ax2.cla()
    # PLot true trajectory
    data = np.array([state.state_vector for state in gt_track])
    ax1.plot(data[:, 0], data[:, 2], 'b-')
    # PLot estimated trajectory
    data = np.array([state.state_vector for state in track.states])
    ax1.plot(data[:, 0], data[:, 2], 'r-')
    # Plot innovation covariance
    plot_cov_ellipse(meas_prediction.covar,
                     meas_prediction.mean, edgecolor='b',
                     facecolor='none', ax=ax1)
    # Plot estimated covariance
    plot_cov_ellipse(track.state.covar[[0, 2], :][:, [0, 2]],
                     track.state.mean[[0, 2], :], edgecolor='r',
                     facecolor='none', ax=ax1)
    # Visualise model weights
    ax2.bar([1,2], prior.weights.ravel())
    plt.pause(0.0001)
    '''

track_error = np.asarray(track_error)
track_error = np.squeeze(track_error)
rmse_tmp = []
for track_error_ in track_error:
    rmse_tmp.append(track_error_ @ track_error_.T)
rmse = np.sqrt(np.mean(np.array(rmse_tmp)))
print("RMSE: " + str(rmse))

RMSE: 0.6351224575313863


In [10]:
get_Q(0.5)

array([[0.16666667, 0.25      , 0.        , 0.        ],
       [0.25      , 0.5       , 0.        , 0.        ],
       [0.        , 0.        , 0.16666667, 0.25      ],
       [0.        , 0.        , 0.25      , 0.5       ]])